# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)
output_data_file = "../output_data/city_weather.csv"

# pull prevously generated and saved output data from csv
weather_df = pd.read_csv(output_data_file)

weather_df


,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed
0,0,Jamestown,US,1609646793,42.10,-79.24,31.30,32.00,100,90,4.70
1,1,Busselton,AU,1609646876,-33.65,115.33,78.24,79.00,41,99,5.01
2,2,Bonavista,CA,1609646878,48.65,-53.11,21.20,21.20,85,100,1.12
3,3,Coxim,BR,1609646879,-18.51,-54.76,77.14,77.14,71,47,4.56
4,4,Tutóia,BR,1609646880,-2.76,-42.27,80.10,80.10,82,82,12.28
...,...,...,...,...,...,...,...,...,...,...,...
579,579,Bushehr,IR,1609647640,28.97,50.84,59.00,59.00,87,0,11.70
580,580,Trabzon Province,TR,1609647642,40.92,39.83,48.20,48.20,87,75,5.82
581,581,Erenhot,CN,1609647643,43.65,111.98,-1.28,-1.28,84,70,3.85
582,582,Sept-Îles,CA,1609647605,50.20,-66.38,16.68,19.40,85,90,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70
# Wind speed less than 10 mph
# Less than or equal to 15 % cloudiness
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# get low temp outliers index
temp_outlier_df = weather_df.loc[weather_df["max temperature"] < 70]
weather_df.drop(weather_df["city_id"][temp_outlier_df.index.tolist()], inplace = True)    

temp_outlier_df = weather_df.loc[weather_df["max temperature"] > 80]
weather_df.drop(weather_df["city_id"][temp_outlier_df.index.tolist()], inplace = True)    
  
wind_outlier_df = weather_df.loc[weather_df["wind speed"] > 10]
weather_df.drop(weather_df["city_id"][wind_outlier_df.index.tolist()], inplace = True)    

cloud_outlier_df = weather_df.loc[weather_df["cloudiness"] >= 15]
weather_df.drop(weather_df["city_id"][cloud_outlier_df.index.tolist()], inplace = True)    
      

vacation_weather_df = weather_df
vacation_weather_df

,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed
27,27,Coronel Oviedo,PY,1609646763,-25.42,-56.45,71.46,71.46,54,0,5.95
37,37,Rikitea,PF,1609646927,-23.12,-134.97,79.11,79.11,68,10,8.84
44,44,Venado Tuerto,AR,1609646938,-33.75,-61.97,74.50,74.50,22,5,9.69
57,57,Mar del Plata,AR,1609646912,-38.00,-57.56,69.42,70.00,60,0,4.70
170,170,Pochutla,MX,1609647101,15.74,-96.47,75.99,75.99,82,0,4.14
174,174,Yendi,GH,1609647107,9.44,-0.01,77.49,77.49,52,0,6.71
189,189,Maragogi,BR,1609647132,-9.01,-35.22,76.44,76.44,82,6,8.61
199,199,Mattru,SL,1609647146,7.62,-11.83,74.30,74.30,94,7,2.46
221,221,Elmina,GH,1609647174,5.09,-1.35,78.04,78.04,88,5,8.37
234,234,Reconquista,AR,1609647191,-29.15,-59.65,71.26,71.26,39,0,8.01


In [6]:
vacation_cities_count = vacation_weather_df["city_id"].nunique()
print(f"Wow! {vacation_cities_count} cities with ideal vacation weather. Let's find a place to stay...")

Wow! 20 cities with ideal vacation weather. Let's find a place to stay...


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
city_id = []
hotel_name = []
hotel_address = []
hotel_rating = []
hotel_lat = []
hotel_lng = []


# find the closest hotel/lodging for each city location
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",    
    "key": g_key        
}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_weather_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotels_response = requests.get(base_url, params=params)
    hotels_json = hotels_response.json()
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    try:
        # print(json.dumps(hotels_json, indent=4, sort_keys=True))
        if ((hotels_json["results"][0]["name"] != "") & (hotels_json["results"][0]["vicinity"] != "") & (hotels_json["results"][0]["rating"] > 0)):
            city_id.append(index)
            hotel_name.append(hotels_json["results"][0]["name"])            
            hotel_address.append(hotels_json["results"][0]["vicinity"])
            hotel_rating.append(hotels_json["results"][0]["rating"])          
          

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

  



Missing field/result... skipping.


In [8]:
# create a data frame for hotel_name, hotel_address, hotel_rating

hotel_dict = {
    "city_id": city_id,
    "Hotel Name": hotel_name,
    "Hotel Address": hotel_address,
    "Hotel Rating": hotel_rating
}

hotel_df = pd.DataFrame(hotel_dict)          
    
hotel_df  


,city_id,Hotel Name,Hotel Address,Hotel Rating
0,27,San Andres Palace,"Doctor Blas Garay, Coronel Oviedo",3.5
1,37,Pension Maro'i,Rikitea,4.0
2,44,La Bianca.,"RN 33 26 de Abril y, Venado Tuerto",4.7
3,57,Gran Hotel Mar del Plata,"Rivadavia 2235, Mar del Plata",3.8
4,170,Hotel Posada San Jose,"Constitución, Centro, San Pedro Pochutla",3.8
5,174,Eyarro Lodge,Yendi,2.8
6,189,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124, s/n - Sitio Carió...",4.7
7,221,Esteem Kofi and Adjoa Guesthouse,"Ayensudo, Elmina. Accra-Takoradi Road at Brenu...",3.1
8,234,Grand Hotel,"General Obligado 883, Reconquista",4.0
9,240,Aldea del Bazar Hotel and Spa,"Boulevard Benito Juárez s/n, Lote 7 Fracc, Bac...",4.3


In [9]:
# print(json.dumps(name_address, indent=4, sort_keys=True))

In [10]:
merge_vacations_df = pd.merge(vacation_weather_df, hotel_df, on="city_id")

# Store 'Lat' and 'Lng' into  locations 
locations = merge_vacations_df[["latitude", "longitude"]].astype(float)
merge_vacations_df

,city_id,city,country,date,latitude,longitude,temperature,max temperature,humidity,cloudiness,wind speed,Hotel Name,Hotel Address,Hotel Rating
0,27,Coronel Oviedo,PY,1609646763,-25.42,-56.45,71.46,71.46,54,0,5.95,San Andres Palace,"Doctor Blas Garay, Coronel Oviedo",3.5
1,37,Rikitea,PF,1609646927,-23.12,-134.97,79.11,79.11,68,10,8.84,Pension Maro'i,Rikitea,4.0
2,44,Venado Tuerto,AR,1609646938,-33.75,-61.97,74.50,74.50,22,5,9.69,La Bianca.,"RN 33 26 de Abril y, Venado Tuerto",4.7
3,57,Mar del Plata,AR,1609646912,-38.00,-57.56,69.42,70.00,60,0,4.70,Gran Hotel Mar del Plata,"Rivadavia 2235, Mar del Plata",3.8
4,170,Pochutla,MX,1609647101,15.74,-96.47,75.99,75.99,82,0,4.14,Hotel Posada San Jose,"Constitución, Centro, San Pedro Pochutla",3.8
5,174,Yendi,GH,1609647107,9.44,-0.01,77.49,77.49,52,0,6.71,Eyarro Lodge,Yendi,2.8
6,189,Maragogi,BR,1609647132,-9.01,-35.22,76.44,76.44,82,6,8.61,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124, s/n - Sitio Carió...",4.7
7,221,Elmina,GH,1609647174,5.09,-1.35,78.04,78.04,88,5,8.37,Esteem Kofi and Adjoa Guesthouse,"Ayensudo, Elmina. Accra-Takoradi Road at Brenu...",3.1
8,234,Reconquista,AR,1609647191,-29.15,-59.65,71.26,71.26,39,0,8.01,Grand Hotel,"General Obligado 883, Reconquista",4.0
9,240,Puerto Escondido,MX,1609647198,15.85,-97.07,77.04,77.04,72,1,6.55,Aldea del Bazar Hotel and Spa,"Boulevard Benito Juárez s/n, Lote 7 Fracc, Bac...",4.3


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in merge_vacations_df.iterrows()]
locations = merge_vacations_df[["latitude", "longitude"]]


In [13]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=4, info_box_content=hotel_info
)

fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))